# Entrega 4 - Redes Neuronales

**Grupo 02:**

| Nombre         | C.I       | Email                        |
|----------------|-----------|------------------------------|
| Santiago Alaniz| 5082647-6 | santiago.alaniz@fing.edu.uy  |
| Bruno De Simone| 4914555-0 | bruno.de.simone@fing.edu.uy  |
| María Usuca    | 4891124-3 | maria.usuca@fing.edu.uy      |



## Objetivo

Considere **[Fashion-MNIST1](https://github.com/zalandoresearch/fashion-mnist/tree/master/data/fashion)**, un conjunto de datos con imágenes de 10 tipos diferentes de artículos de la empresa de vestimenta Zalando.

Este laboratorio busca desarrollar y optimizar un clasificador basado en redes neuronales para el dataset `Fashion-MNIST` de Zalando. Inicialmente, se establecerá una arquitectura base de red neuronal feedforward con parámetros específicos.

Posteriormente, se experimentará con tres arquitecturas adicionales para mejorar la clasificación. Al identificar el modelo más prometedor, se aplicarán técnicas de regularización y se comparará su rendimiento con benchmarks existentes. 

Finalmente, se identificarán las imágenes más desafiantes para el clasificador. 

Todo el desarrollo se realizará utilizando `PyTorch`, los resultados y análisis se presentarán en esta Jupyter Notebook.

In [ ]:
!pip3 install torch torchvision

##  Diseño

### Carga de datos

Al dia de la fecha, dada la popularidad del dataset, muchas librerias han incorporado la carga de `Fashion-MNIST` como parte de su API.

`PyTorch` no es la excepción, la incluye en su libreria para datasets de visión artificial `torchvision`.

Particionamos los datos en dos conjuntos con `torch.utils.data.random_split` de la siguiente manera:

- `train` para entrenar el modelo.
- `eval` para evaluar el modelo.
- `test` para evaluar el modelo final.

Ademas, por cuestiones de reproducibilidad, se particiona el conjunto de entrenamiento en `train` y `val` con un generador determinado por una semilla.

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor

TEST_SIZE = 0.5
SEED_NUMBER = 42069

train = datasets.FashionMNIST('./data', train=True, download=True, transform= ToTensor())
test = datasets.FashionMNIST('./data', train=False, download=True, transform= ToTensor())

deterministic_generator = torch.Generator()
deterministic_generator.manual_seed(SEED_NUMBER)

test, eval = torch.utils.data.random_split(
    test, 
    [int(len(test)*(1-TEST_SIZE)), int(len(test)*TEST_SIZE)],
    generator=deterministic_generator
)

In [ ]:
import matplotlib.pyplot as plt

LABLES = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3

for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train), size=(1,)).item()
    img, label = train[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(LABLES[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
    
plt.show()

### Preprocesamiento de datos

Para este laboratorio optamos por no relizar nosotros mismos el preprocesamiento, sino valernos de las herramientas que provee `Pytorch`. En particular [DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) nos permite preparar los datos para el entrenamiento de la red neuronal.

`DataLoader` actua como [wrapper](https://en.wikipedia.org/wiki/Wrapper_function)  de un dataset, permitiendo iterar sobre el mismo en batches de tamaño configurable. Además, permite realizar transformaciones sobre los datos, en nuestro caso, transformamos las imagenes a tensores.

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

train_dataloader = DataLoader(train, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test, batch_size=BATCH_SIZE)
eval_dataloader = DataLoader(eval, batch_size=BATCH_SIZE)

for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break


### Algoritmo

```
    a) Construya un clasificador base

    - i) Defina un red neuronal feedforward base con una capa oculta de 32 unidades y que utilice la función sigmoide como activación, que devuelva, para una imagen de entrada, una distribución de probabilidad sobre las categorías, y permita asignarle la categoría más probable.
```



### Evaluación

```
- ii) Utilizando el conjunto de entrenamiento provisto, entrene a la red construida en el paso anterior durante 10 épocas
- iii) Evalúe el rendimiento del clasificador construido sobre un conjunto de validación, utilizando descenso por gradiente estocástico y una tasa de aprendizaje de 0.01.
- iv) Reporte gráficamente la evolución de la pérdida en el conjunto de entrenamiento y de la accuracy sobre el conjunto de validación en función de las iteraciones.
```

## Experimentación

```
b) Proponga tres arquitecturas adicionales que busquen mejorar los resultados, modificando la cantidad de unidades, la cantidad de capas ocultas, y/o diferentes funciones de activación. Para cada una, evalúe su rendimiento sobre un conjunto de validación, con diferentes valores de tasa de aprendizaje. 

c) A partir del mejor modelo obtenido en b), sugiera y aplique algún mecanimo de regularización y vuelva a evaluar sobre el conjunto de validación, igual que en el paso anterior.

d) Con el mejor modelo obtenido luego de los pasos anteriores, evalúe su performance sobre el conjunto de evaluación utilizando accuracy, precision, recall y medida F1 para cada una de las clases. 

Construya la matriz de confusión. Comente los resultados y compare con los reportados en el sitio del dataset. 

e) Muestre las diez instancias del conjunto de evaluación más “difíciles” para el clasificador construido, utilizando como medida la entropía. Comente los resultados.
```

## Conclusión

## Bibliografia.

- [Tutorial de PyTorch](https://pytorch.org/tutorials/beginner/basics/intro.html)
- [Fashion-MNIST1](https://github.com/zalandoresearch/fashion-mnist/tree/master/data/fashion)